In [1]:
# Library Installation
!pip install ultralytics
!pip install deep-sort-realtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 711.5 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [ ]:
# Import libraries and packages
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
# Path to pretrained weight '.pt' file
WEIGHTS_PATH = r""

# Define paths for input video and output video
VIDEO_PATH = r""
OUTPUT_VIDEO_PATH = r""
COUNT_FILE_PATH = r""

# Tuneable Parameters
# Define Region of Interest (ROI) coordinates
ROI_PX_COORDINATES = np.array([[64, 267], [89, 383], [307, 322], [197, 273]], np.int32) # Define ROI coordinates from get_ROI.py here
# Frame skip rate for reducing computation
FRAME_SKIP_RATE = 1
# Confidence threshold for model prediction
PREDICTION_CONFIDENCE_THRESHOLD = 0.4


In [13]:
# Loading the model
model = YOLO(WEIGHTS_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

target_size = (640, 640)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, target_size)

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3)
object_counts = defaultdict(set)

def is_within_roi(box, ROI_PX_COORDINATES):
    """
    Check if the center of the bounding box is within the ROI polygon.
    """
    x1, y1, x2, y2 = map(int, box)
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2
    return cv2.pointPolygonTest(ROI_PX_COORDINATES, (center_x, center_y), False) >= 0

def resize_with_padding(image, target_size):
    """
    Resize the image with padding to match the target size.
    """
    original_h, original_w = image.shape[:2]
    target_h, target_w = target_size

    scaling_factor = min(target_w / original_w, target_h / original_h)
    new_w = int(original_w * scaling_factor)
    new_h = int(original_h * scaling_factor)

    resized_image = cv2.resize(image, (new_w, new_h))

    new_image = np.zeros((target_h, target_w, 3), dtype=np.uint8)
    new_image.fill(0)

    top = (target_h - new_h) // 2
    left = (target_w - new_w) // 2

    new_image[top:top + new_h, left:left + new_w] = resized_image

    return new_image

frame_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    if frame_counter % FRAME_SKIP_RATE != 0:
        continue

    # Resize the frame with padding
    frame = resize_with_padding(frame, target_size)

    # Perform object detection using YOLO model
    results = model(frame)
    detections = []
    classes = []

    # Extract bounding boxes, confidences, and class labels
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        cls_array = result.boxes.cls.cpu().numpy()

        for box, confidence, cls in zip(boxes, confidences, cls_array):
            if confidence > PREDICTION_CONFIDENCE_THRESHOLD:
                x1, y1, x2, y2 = map(int, box)
                label = model.names[int(cls)]

                if is_within_roi(box, ROI_PX_COORDINATES):
                    width = x2 - x1
                    height = y2 - y1
                    detections.append(([x1, y1, width, height], float(confidence), int(cls)))
                    classes.append(int(cls))
                    color = (0, 255, 0)  # Green bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Update the tracker with current frame's detections
    tracks = tracker.update_tracks(detections, frame=frame, others=classes)

    # Draw tracking information and update object counts
    for track, cls in zip(tracks, classes):
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        label = model.names[cls]

        # Draw bounding box and track ID
        color = (255, 0, 0)  # Blue for tracking
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 1)
        cv2.putText(frame, f'{label}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Update the count if the object is within ROI for the first time
        if is_within_roi((x1, y1, x2, y2), ROI_PX_COORDINATES):
            if track_id not in object_counts[label]:
                object_counts[label].add(track_id)
                
    cv2.polylines(frame, [ROI_PX_COORDINATES], isClosed=True, color=(255, 0, 0), thickness=1)
    out.write(frame)
cap.release()
out.release()

# Print the object counts summary
object_counts_summary = {label: len(ids) for label, ids in object_counts.items()}
print("Object Counts:", object_counts_summary)

# Save the object counts to a text file
with open(COUNT_FILE_PATH, 'w') as f:
    for label, count in object_counts_summary.items():
        f.write(f'{label}: {count}\n')



0: 416x416 6 cars, 1 two-wheeler, 8 auto-rickshaws, 10.4ms
Speed: 1.9ms preprocess, 10.4ms inference, 3.2ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 6 cars, 1 two-wheeler, 8 auto-rickshaws, 10.4ms
Speed: 2.3ms preprocess, 10.4ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 5 cars, 1 two-wheeler, 9 auto-rickshaws, 10.5ms
Speed: 2.4ms preprocess, 10.5ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 5 cars, 1 two-wheeler, 9 auto-rickshaws, 10.9ms
Speed: 2.6ms preprocess, 10.9ms inference, 1.8ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 4 cars, 1 two-wheeler, 8 auto-rickshaws, 14.2ms
Speed: 2.4ms preprocess, 14.2ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 2 cars, 1 two-wheeler, 9 auto-rickshaws, 11.0ms
Speed: 4.0ms preprocess, 11.0ms inference, 1.8ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 4 cars, 1 two-wheeler, 7 auto-rickshaws, 10.4m